In [ ]:
#Nur für Colab
#Datei hochladen
from google.colab import files

uploaded = files.upload()

Saving Symptom2Disease.csv to Symptom2Disease.csv


In [ ]:
#Für Billy
from google.colab import files

billys_csv = files.upload()

Saving Data1.csv to Data1.csv


# 1 Problemstellung

- Problemart: supervised Multi-Class Single-Label Text Classification
- Supervised = Model wird mit Datensätzen trainiert, bei denen das Label bereits bekannt ist
- Classification = Zuordnung zu diskreten Werten
- Multiclass Single-Label = mehrere Labels, jedem Sample kann nur ein Label zugewiesen werden

Andere Arten:
- Multi-Label Classification (mehrere Labels, jedem Sampel können mehrere Labels zugewiesen werden)
- Binary Classification (z.B. Sentiment-Analysis, nur zwei Label, z.B. positiv und negativ)

# 2 Methoden
_https://en.wikipedia.org/wiki/Multiclass_classification_<br>
_https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=a546f2c88c588a2a46c054f67b39a3ebefdae694_<br>
_https://www.ijcsit.com/docs/Volume%204/Vol4Issue4/ijcsit2013040408.pdf_

## 2.1 Umwandeln in binäre Klassifikation
= Problem wird auf mehrere binäre Klassifikationen zurückgeführt

### 2.1.1 One-vs.-all/One-vs.-rest
_https://youtu.be/ZvaELFv5IpM_
<figure>
    <img src="https://www.cc.gatech.edu/classes/AY2016/cs4476_fall/results/proj4/html/jnanda3/svm.png" alt="One-vs.-all Algorithmus" style="width: 400px;"/>
    <br>
</figure>
<small>Quelle: https://www.cc.gatech.edu/classes/AY2016/cs4476_fall/results/proj4/html/jnanda3/index.html</small>

- es wird pro Klasse ein binärer Classifier (z.B. eine logistische Regression oder SVM) trainiert nach dem Prinzip "einer gegen alle"
    - Beispiel: rot, grün, blau und gelb
        - rot vs. nicht rot
        - grün vs. nicht grün
        - blau vs. nicht blau
        - gelb vs. nicht gelb
- dann werden auf jeden Punkt im Datensatz alle Classifier angewendet
- der Classifier, der die größte Confidence hat, wird gewählt, um den Punkt zu klassifizieren
- wird bei großer Anzahl an Klassen ineffizient

### 2.1.2 One-vs.-one
_https://machinelearningmastery.com/one-vs-rest-and-one-vs-one-for-multi-class-classification/_
<figure>
    <img src="https://image.slidesharecdn.com/linearmodelsandmulticlassclassification2-170312171304/75/linear-models-and-multiclass-classification-25-2048.jpg?cb=1667658416" alt="One-vs.-one Algorithmus" style="width: 400px;"/>
    <br>
</figure>
<small>Quelle: https://www.slideshare.net/Paxcel/binary-and-multi-class-strategies-for-machine-learning</small>

- Problem wird in binäre Classifier aufgeteilt nach dem Prinzip "jede Klasse gegen jede"
    - Beispiel: rot, grün, blau und gelb
        - rot vs. grün
        - rot vs. blau
        - rot vs. gelb
        - grün vs. blau
        - grün vs. gelb
        - blau vs. gelb
- jeder Classifier wird nur mit den Datensätzen trainiert, die einer der beiden Klassen angehören
- Anzahl der binären Klassifikationsprobleme: (N * (N-1))/2
- nach Scoring-System wird am Ende eine Klasse bestimmt

## 2.2 Erweitern der binären Klassifikation
= Classifier, die eigentlich für binäre Klassifikation gedacht sind, aber für multiclass erweitert werden

### 2.2.1 Neuronales Netz
_https://developers.google.com/machine-learning/crash-course/multi-class-neural-networks/video-lecture_
- Neuronales Netz nach One-vs.-all: 
    - NN für binäre Klassifikation hat üblicherweise einen Output-Knoten, der binär klassifiziert, stattdessen neuronales Netz mit so vielen Output-Knoten, wie es Klassen gibt
    - die Wahrscheinlichkeiten der Output-Knoten sollen addiert 1 ergeben (-> Softmax als Loss-Funktion/Optimierungsfunktion)
- Nachteil: verlangt sehr viel mehr Rechenzeit und -power als andere Herangehensweisen

In [ ]:
import pandas as pd
import io
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM

# Load the dataset
data = pd.read_csv(io.BytesIO(billys_csv['Data1.csv']))
data.pop("id")

# Convert diagnosis column to integers
diagnosis_dict = {diagnosis: i for i, diagnosis in enumerate(data['label'].unique())}
data['label'] = data['label'].apply(lambda x: diagnosis_dict[x])

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)

# Convert the diagnosis labels into a one-hot encoding format
num_classes = len(data['label'].unique())
y_train = np.eye(num_classes)[y_train]
y_test = np.eye(num_classes)[y_test]

# Tokenize the symptoms and convert them into sequences
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to ensure that they are all the same length
max_len = 100
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_len)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len)

# Build the neural network model
embedding_size = 128
model = Sequential()
model.add(Embedding(max_words, embedding_size, input_length=max_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))  #bei mehreren Layers noch zusätzlich return_sequences=True eintragen
#model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
model.fit(X_train_padded, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_padded, y_test))

# Evaluate the model on the testing set
loss, accuracy = model.evaluate(X_test_padded, y_test, batch_size=batch_size)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

NameError: ignored

### 2.2.2 K-nearest-neighbor
_https://towardsdatascience.com/multiclass-classification-using-k-nearest-neighbours-ca5281a9ef76_
- Abstand des zu klassifizierenden Punktes zu allen seinen Nachbarn wird berechnet
- Nachbarn werden nach Abstand sortiert
- Punkt wird in die Klasse eingeordnet, der die meisten seiner k-nächsten Nachbarn angehören
- Funktioniert besser bei kleinen Datensätzen, wird sonst ineffizient

In [ ]:
#Vor dem Ausführen der Zelle muss ganz oben einmal die csv-Datei hochgeladen werden!
#Beispiel-Implementierung von K-nearest-neighbor
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import io

#Datei einlesen
data = pd.read_csv(io.BytesIO(uploaded['Symptom2Disease.csv']))
data.pop("id")
data = data.sample(frac = 1)

#Aufteilen in Trainings- und Testdaten
X = data["text"]
y = data["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)

#Vektorisieren
cv = CountVectorizer()
X_train_counts = cv.fit_transform(X_train)
X_test_counts = cv.transform(X_test)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

#Trainieren
knn = KNeighborsClassifier()

knn.fit(X_train_tfidf, y_train)

#Vorhersagen der Testdaten
predicted = knn.predict(X_test_tfidf)

#Auswertung
print(metrics.classification_report(y_test, predicted))

                                 precision    recall  f1-score   support

                           Acne       1.00      1.00      1.00        16
                      Arthritis       0.91      1.00      0.95        20
               Bronchial Asthma       1.00      0.81      0.90        16
           Cervical spondylosis       0.95      1.00      0.98        20
                    Chicken pox       0.64      0.93      0.76        15
                    Common Cold       0.93      1.00      0.97        14
                         Dengue       0.71      0.89      0.79        19
          Dimorphic Hemorrhoids       1.00      1.00      1.00        19
               Fungal infection       0.95      1.00      0.98        21
                   Hypertension       0.92      0.92      0.92        13
                       Impetigo       1.00      0.83      0.91        12
                       Jaundice       0.92      1.00      0.96        11
                        Malaria       0.90      1.

### 2.2.3 Naive Bayes
_https://youtu.be/EGKeC2S44Rs_
- Familie von Algorithmen
- Stellen die "naive" Annahme, dass die Wörter alle voneinander unabhängig sind
- Ablauf:
    - Datensätze werden vektorisiert (Count Vectorizer):
        - Länge des Vektors = Anzahl der Worte im Vokabular
        - Vektor drückt aus, wie oft jedes Wort des Vokabulars im Datensatz enthalten ist
    - Aus den Trainingsdaten wird die Wahrscheinlichkeit für jede Klasse abgeleitet
    - für jede Klasse wird die Wahrscheinlichkeit von jedem Wort berechnet

In [ ]:
#Vor dem Ausführen muss ganz oben einmal die Datei hochgeladen werden!
#Beispiel für Implementierung von Naive Bayes Classifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
import io

#Datei einlesen
data = pd.read_csv(io.BytesIO(uploaded['Symptom2Disease.csv']))
data.pop("id")
data = data.sample(frac = 1)

#Aufteilen in Trainings- und Testdaten
X = data["text"]
y = data["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)

#Pipeline erstellen mit Vektorisierung, Transformation und Naive Bayes Classifier
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                     ])

#Trainieren
text_clf.fit(X_train, y_train)

#Vorhersagen der Testdaten
predicted = text_clf.predict(X_test)

#Auswertung
print(metrics.classification_report(y_test, predicted))

                                 precision    recall  f1-score   support

                           Acne       1.00      1.00      1.00        18
                      Arthritis       0.82      1.00      0.90        14
               Bronchial Asthma       0.89      1.00      0.94        16
           Cervical spondylosis       1.00      0.94      0.97        17
                    Chicken pox       1.00      0.94      0.97        18
                    Common Cold       0.94      0.94      0.94        18
                         Dengue       0.86      0.86      0.86        14
          Dimorphic Hemorrhoids       0.88      1.00      0.93        14
               Fungal infection       0.74      1.00      0.85        14
                   Hypertension       1.00      1.00      1.00        12
                       Impetigo       0.88      1.00      0.93        14
                       Jaundice       0.95      1.00      0.97        19
                        Malaria       0.90      1.

### 2.2.4 Support Vector Machine
_https://www.baeldung.com/cs/svm-multiclass-classification#multiclass-classification-using-svm_
- Ursprünglich designed für binäre Klassifikation, aber auch anwendbar auf Multi-Class Probleme
- Grundsätzliche Vorgehensweise bei binärer Klassifikation:
    - Ziel: Linie (bzw. Ebene) finden, die die Punkte der beiden Klassen am besten trennt (maximiert den Abstand zu den Punkten beider Klassen)
    - Die Datensätze, die der Linie am nächsten sind, sind die Support-Vectors
    - Abstand der Support-Vectors zur Linie ist die Margin, die maximiert werden soll
    - Die Berechnung der Trennlinie wird mit eine Kernel-Funktion durchgeführt
    - Es gibt verschiedene Kernel-Funktionen, deren Hyperparameter beim Tuning angepasst werden
- Zwei Herangehensweisen: 
    - One-to-one
    - One-to-rest
- Verfügbar in sklearn
- Vorteile:
    - Speichereffizient
    - anpassbar (spezifische Kernel Functions auswählbar)
    - kommt mit kleinem Trainigsdatensatz aus

In [ ]:
#Vor dem Ausführen muss ganz oben einmal die Datei hochgeladen werden!
#Beispiel für die Implementierung von linear Support-Vector-Machine
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
import io

#Datei einlesen
data = pd.read_csv(io.BytesIO(uploaded['Symptom2Disease.csv']))
data.pop("id")
data = data.sample(frac = 1)

#Aufteilen in Trainings- und Testdaten
X = data["text"]
y = data["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)

#Pipeline mit Vektorisierung, Transformation und SVM
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LinearSVC()),
                     ])

#Trainieren
text_clf.fit(X_train, y_train)

#Vorhersagen der Testdaten
predicted = text_clf.predict(X_test)

#Auswertung
print(metrics.classification_report(y_test, predicted))

                                 precision    recall  f1-score   support

                           Acne       1.00      1.00      1.00        17
                      Arthritis       1.00      1.00      1.00        17
               Bronchial Asthma       1.00      1.00      1.00        11
           Cervical spondylosis       1.00      1.00      1.00        21
                    Chicken pox       1.00      0.93      0.96        14
                    Common Cold       0.95      1.00      0.97        18
                         Dengue       0.87      1.00      0.93        13
          Dimorphic Hemorrhoids       1.00      1.00      1.00        18
               Fungal infection       1.00      1.00      1.00        15
                   Hypertension       1.00      1.00      1.00        15
                       Impetigo       1.00      1.00      1.00        23
                       Jaundice       1.00      1.00      1.00        15
                        Malaria       1.00      1.

### 2.2.5 Decision Trees
_https://www.youtube.com/watch?v=ZVR2Way4nwQ_
- Model entwickelt einen Entscheidungsbaum anhand der Trainingsdaten
- Aufbau des Entscheidungsbaums:
    - Alle Datensätze sind in der Root
    - Alle Blätter enthalten nur noch Datensätze von einer Klasse
    - Model vergleicht alle möglichen Aufteilungen (Splits)
    - Model sucht Split so aus, dass die Entropie in einem Kindknoten möglichst gering ist -> und der Informationgain möglichst groß ist
- neue Datensätze werden mit Hilfe des Entscheidungsbaums in die entsprechende Klasse eingeordnet

In [ ]:
#Vor dem Ausführen muss ganz oben die Datei einmal hochgeladen werden!
#Beispiel für die Implementierung von Decision Trees
import pandas as pd
import io
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#Datei einlesen
data = pd.read_csv(io.BytesIO(uploaded['Symptom2Disease.csv']))
data.pop("id")
data = data.sample(frac = 1)

#Aufteilen in Trainings- und Testdaten
X = data["text"]
y = data["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)

#Pipeline mit Vektorisierung, Transformation und Decision Tree
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', tree.DecisionTreeClassifier()),
                     ])

#Trainieren
text_clf.fit(X_train, y_train)

#Vorhersagen der Testdaten
predicted = text_clf.predict(X_test)

#Auswertung
print(metrics.classification_report(y_test, predicted))

                                 precision    recall  f1-score   support

                           Acne       1.00      1.00      1.00        24
                      Arthritis       1.00      1.00      1.00        14
               Bronchial Asthma       0.85      0.85      0.85        20
           Cervical spondylosis       0.92      0.92      0.92        13
                    Chicken pox       0.27      0.19      0.22        16
                    Common Cold       0.52      0.79      0.63        14
                         Dengue       0.36      0.50      0.42        16
          Dimorphic Hemorrhoids       1.00      1.00      1.00        15
               Fungal infection       0.92      0.67      0.77        18
                   Hypertension       1.00      0.73      0.84        11
                       Impetigo       0.76      0.76      0.76        17
                       Jaundice       1.00      1.00      1.00        16
                        Malaria       1.00      0.

###2.2.6 Gradient Boost

https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/

* basiert auf Ensemble Methoden
* kombiniert mehrere schwache Lernalgorithmen zu einer starken; jeder Algorithmus wird nacheinander trainiert und versucht den schwächeren zu verbessern


*   beginnt mit Basismodel von beliebigen Algorithmus, der für Klassifikation oder Regression geeignet ist
    
    - neues Model wird erstellt und korrigiert die Fehler des Basismodels; Gradientenabstieg wird verwendet um das neue Modell zu trainieren

    - neues trainiertes Modell wird dem Basismodell hinzugefügt, um ein neues Essemble Modell zu erstellen -> das wird oft wiederholt
  


*   sehr leistungsfähiger Algorithmus und gut für große Datensätze, vor Allem weil das Modell durch die ganzen Wiederholungen sehr präzise wird

    - ist aber auch sehr rechenintensiv












In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import io

# Einlesen des Datensatzes als CSV-Datei
df = pd.read_csv(io.BytesIO(uploaded['Symptom2Disease.csv']))
df.pop("id")

# Aufteilen der Features und Labels
X = df['text']
y = df['label']

# Aufteilen des Datensatzes in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#Pipeline mit Vektorisierung, Transformation und Gradient Boost Classifier
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('gb_clf', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)),
                     ])

# Trainieren des Modells
text_clf.fit(X_train, y_train)

# Vorhersage auf Testdaten machen
y_pred = text_clf.predict(X_test)

# Berechnen der Genauigkeit
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.89


## 2.3 Hierarchische Klassifikation
= Aufteilung in Ober- und Unterklassen in einem Baum, für die Aufteilung an den einzelnen Knoten wird meist ein binärer Classifier genutzt

# 3 Persistenz
*https://scikit-learn.org/stable/model_persistence.html*
- wichtig bei Text-Klassifizierung: sowohl das Model, als auch der/die Vectorizer müssen gespeichert werden
- es muss außerdem sichergestellt werden, dass das Model in die gleiche scikit-learn Version geladen wird, in der es auch gespeichert wurde

## 3.1 Python-spezifische Serialisierung

- unterstützen alle auch das Speichern gesamter Pipelines

### 3.1.1 Speichern mit Pickle

In [ ]:
#In colab wird das Model nur in dem Sitzungsspeicher abgespeichert, also ist das Model nur für die aktuelle Sitzung persistent (kann aber natürlich aus Colab runtergeladen oder ein anderes hochgeladen werden)
#Aber wenn man es in JupyterNotebook o.Ä. benutzt wird es lokal im selben Verzeichnis gespeichert
import pickle

#Speichern
with open('file.pkl', 'wb') as f:
  pickle.dump(model, f)

#Laden
with open('file.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

### 3.1.2 Speichern mit joblib

- effizienter als Pickle, besonders bei großen Arrays

In [ ]:
from joblib import dump, load

#Speichern
dump(model, 'file.joblib')

#Laden
loaded_model = load('file.joblib')

### 3.1.3 Speichern mit skops
*https://skops.readthedocs.io/en/stable/persistence.html*

- sicherer als joblib und Pickle
- lädt nur Models, die keine "untrusted types" enthalten

In [ ]:
import skops.io as sio

#Speichern
sio.dump(model, 'file.skops')

#Überprüfung der untrusted types
unknown_types = sio.get_untrusted_types(file="file.skops")
print(unknown_types)

#Laden des Models nach Überprüfung der untrusted types
loaded_model = sio.load("file.skops", trusted=unknown_types)

## 3.2 Kompatible Formate

- wenn ein Model über unterschiedliche Architekturen und Umgebungen hinaus genutzt werden soll, sollte das Model am besten in eines der "interoperable formats" exportiert werden
- z.B. wenn die Prediction in einer anderen Umgebung gemacht werden soll als das Training

### 3.2.1 Open Neural Network Exchange (ONNX)
*http://onnx.ai/sklearn-onnx/*
- binäre Serialisierung des Models
- Vorgaben/Einschränkungen:
  - Für das Konvertieren muss der vektorisierte Datensatz von einer "sparse matrix" in eine "dense matrix" umgewandelt werden.
  - Nachteil dabei ist, dass sparse matrices effizienter (besonders was den Speicher angeht) sind als dense matrices.
  - Außerdem kann das Konvertieren unter Umständen zu einem Verlust bei der accuracy führen, wird aber bei unserer Problemstellung und unseren Models vermutlich kaum eine Rolle spielen.
  - Kann aber ausgeglichen werden, indem das Model auf die Verwendung von dense matrices getuned wird
- Alle Schritte von Vektorisierung bis eigentliches Model müssen für ein beständiges Output konvertiert werden
  - dabei ist es leider nicht so einfach, die alle in einem Model zu speichern
  - deshalb müssen sie vermutlich alle einzeln konvertiert werden, das bedeutet, dass besonders sorgfältig darauf geachtet werden muss, dass Reihenfolge und Formate nach dem Laden eingehalten werden
- Das Konvertieren von gesamten Pipelines ist auch möglich, aber dann ist meist immer noch eine Umwandlung der sparse Matrix in eine dense Matrix nötig, was bei einer Pipeline nur schwer umsetzbar ist


In [ ]:
from onnx import helper, TensorProto, numpy_helper
import onnx
import skl2onnx
import onnxruntime as rt
from onnxconverter_common.data_types import StringTensorType, Int64TensorType, FloatTensorType

#Beispiel für das Transformieren eines Count Vectorizers in ein dense numpy array
X_train_np = X_train_tfidf.toarray()
X_test_np = X_test_tfidf.toarray()

#Vectorizer konvertieren
initial_type = [('input', StringTensorType([None,]))]
onnx_cv = skl2onnx.convert_sklearn(cv, initial_types=initial_type)

#Transformer konvertieren
initial_type = [('input', FloatTensorType([None, X_train_np.shape[1]]))]
onnx_tfidf = skl2onnx.convert_sklearn(tfidf_transformer, initial_types=initial_type)

#Model konvertieren
initial_type = [('input', FloatTensorType([None, X_train_np.shape[1]]))]
onnx_model = skl2onnx.convert_sklearn(model, initial_types=initial_type)

#konvertierte Models speichert
onnx.save_model(onnx_cv, "cv_model.onnx")
onnx.save_model(onnx_tfidf, "tfidf_model.onnx")
onnx.save_model(onnx_model, "model.onnx")

#Laden der Models
model = rt.InferenceSession('knn_model.onnx')

#Ausführen des Models
output = model.run(None, {'input': X_test_np})

### 3.2.2 Predicitve Model Markup Language (PMML)
*https://pypi.org/project/pypmml/*

- nicht alle Algorithmen sind transformierbar (z.B. TF-IDF)
- unkomplizierte Transformation mit Hilfe von Pipelines

In [ ]:
from sklearn2pmml import PMMLPipeline
from sklearn2pmml import sklearn2pmml
import io
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from pypmml import Model

#Daten einlesen
data = pd.read_csv(io.BytesIO(uploaded['Symptom2Disease.csv']))
data.pop("id")
data = data.sample(frac = 1)

#Daten aufteilen
X = data["text"]
y = data["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)

#Pipeline
pipeline = PMMLPipeline([
    ('vect', CountVectorizer()),
    ('clf', KNeighborsClassifier())
])

pipeline.fit(X_train, y_train)

#konvertieren zu pmml
sklearn2pmml(pipeline, "pipeline.pmml")

#Model laden
loaded_pipeline = Model.load("pipeline.pmml")

#predicted = loaded_pipeline.predict(X_test)

# 4 Vergleich der Modelle

**Neuronales Netz:** Hohe Effizienz und Rechenleistung aufgrund der parallelen Verarbeitung von Daten durch mehrere Schichten. Schwer zu interpretieren und erfordert viel Training und Tuning, um Überanpassung zu vermeiden. (Überanpassung bedeutet, dass sich ein Modell zu sehr an die Trainingsdaten angepasst hat, und mit neuen Daten stark überfordert wäre.)

/

**K-nearest-neighbor:** Niedrige Effizienz und Rechenleistung, da alle Datenpunkte zur Vorhersage herangezogen werden müssen. Das Modell ist leicht zu interpretieren, aber empfindlich gegenüber Abweichungen bzw. Ungenauigkeit der Datenwerte.

/

**Naive Bayes:** Hohe Effizienz und niedrige Rechenleistung, da nur einfache statistische Berechnungen durchgeführt werden müssen. Das Modell ist einfach zu interpretieren und gut anpassungsfähig, aber auch empfindlich gegenüber Abweichungen.

/

**Support Vector Machine:** Hohe Effizienz und Rechenleistung, da nur eine kleine Anzahl von Trainingsdaten benötigt wird. Das Modell ist jedoch schwer zu interpretieren und stark abhängig von der Wahl der Kernel- und Hyperparameter, was es schwieriger macht ein Modell zu trainieren, da die Leistung davon abhängig ist.

/

**Decision Trees:** Hohe Effizienz, aber niedrige Rechenleistung aufgrund der Baumstruktur und der Entscheidungsregeln. Das Modell ist leicht zu interpretieren und kann gut mit Abweichungen umgehen, aber empfindlich gegenüber Überanpassung.

/

**Gradient Boost:** Hohe Effizienz und Rechenleistung, da die Entscheidungsregeln iterativ durchgeführt werden. Das Modell ist jedoch schwer zu interpretieren und empfindlich gegenüber Überanpassung.

/

**FAZIT:** Für große und komplexe Datensätze, würde sich wahrscheinlich ein neurolaes Netz, oder ein Gradient Boost am Besten eignen.









# 5 Glossar

### Dimension eines Problems
= Dimension der Matrix der Input-Daten

### Entscheidungsfunktion/Zielfunktion/Optimierungsfunktion
= Algorithmus, der Trainingsdaten auswertet und entscheidet, ob eine Verbesserung oder Verschlechterung stattgefunden hat

### Kernel Functions
= werden bei SVM verwendet, um die Input-Daten für das Processing in die notwendige Form zu transformieren

### Precision
= ist das Maß für den Anteil an vorhergesagten positiven Fällen, die auch tatsächlich positiv sind, demensprechend ist eine hohe precision, eine hohe korrekte Vorhersage des Modells

### Recall 
= ist das Maß für die möglichst wichtigsten/relevantesten erfassten Fälle des Modells, ein hoher Recall Wert bedeuted dementsprechend dass das Modell nur wenige relevante Fälle übersehen hat

### F1-Score
= der F1-Score ist quasi eine Gewichtung aus Recall und Precision, und zeigt wie gut das Modell insgesamt abschneidet

### Odds-Ratio
= ist ein Maß, welches verwendet wird um einen Zusammenhang zwischen Variablen zu quantifizieren - zeigt wie sich die Wahrscheinlichkeit ändert, wenn etwas mit oder ohne spezielle Bedingung erfüllt wird

### Macro AVG
- durchschnittswert, der keine Ungleichheit in der Größe der einzelnen Klassen berücksichtigt

### Weighted AVG
- durchschnittswert, der die Ungleichheit berücksichtigt

### Epoch

### Batch-Size

### Embedding-Size